# Article Editor

Use this notebook to view and edit evergreen (and trending) articles in the database.

## Workflow:
1. Run the setup cell
2. List articles to find the one you want to edit
3. Load the article
4. Edit the content in the markdown cell
5. Save changes back to the database

In [ ]:
# Setup
import sqlite3
import json
from datetime import datetime
from IPython.display import Markdown, display, HTML

DB_PATH = "articles.db"

def get_connection():
    """Get a database connection with row factory."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

print(f"Database: {DB_PATH}")
print("Setup complete!")

## 1. List Articles

View all articles or filter by type.

In [ ]:
def list_articles(article_type=None, limit=20):
    """
    List articles from the database.
    
    Args:
        article_type: 'evergreen', 'trending', or None for all
        limit: Maximum number of articles to show
    """
    conn = get_connection()
    cursor = conn.cursor()
    
    if article_type:
        cursor.execute("""
            SELECT id, article_title, article_type, topic, created_at, updated_at
            FROM articles
            WHERE article_type = ?
            ORDER BY created_at DESC
            LIMIT ?
        """, (article_type, limit))
    else:
        cursor.execute("""
            SELECT id, article_title, article_type, topic, created_at, updated_at
            FROM articles
            ORDER BY created_at DESC
            LIMIT ?
        """, (limit,))
    
    articles = cursor.fetchall()
    conn.close()
    
    if not articles:
        print("No articles found.")
        return []
    
    # Display as formatted table
    print(f"{'ID':>4} | {'Type':>10} | {'Created':>10} | Title")
    print("-" * 80)
    
    for a in articles:
        title = a['article_title'][:45] + "..." if len(a['article_title']) > 45 else a['article_title']
        created = a['created_at'][:10] if a['created_at'] else 'Unknown'
        a_type = a['article_type'] or 'trending'
        print(f"{a['id']:>4} | {a_type:>10} | {created} | {title}")
    
    print(f"\nTotal: {len(articles)} articles")
    return [dict(a) for a in articles]

# List evergreen articles
print("=== Evergreen Articles ===")
evergreen = list_articles(article_type='evergreen')

print("\n=== Trending Articles ===")
trending = list_articles(article_type='trending', limit=10)

## 2. Load Article for Editing

Set the `ARTICLE_ID` to the article you want to edit, then run the cell.

In [ ]:
# ============================================
# CHANGE THIS to the article ID you want to edit
ARTICLE_ID = 1
# ============================================

def load_article(article_id):
    """Load a full article by ID."""
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM articles WHERE id = ?", (article_id,))
    row = cursor.fetchone()
    conn.close()
    
    if not row:
        print(f"Article {article_id} not found!")
        return None
    
    return dict(row)

# Load the article
article = load_article(ARTICLE_ID)

if article:
    print(f"Loaded Article ID: {article['id']}")
    print(f"Title: {article['article_title']}")
    print(f"Type: {article.get('article_type', 'trending')}")
    print(f"Topic: {article['topic']}")
    print(f"Created: {article['created_at']}")
    print(f"Updated: {article.get('updated_at', 'Never')}")
    print(f"Content length: {len(article['article_content'])} characters")
    print("\n" + "=" * 60 + "\n")
    
    # Store original for comparison
    original_title = article['article_title']
    original_content = article['article_content']

## 3. View Article Content

View the current article content rendered as markdown.

In [ ]:
# View the article content as rendered markdown
if article:
    display(Markdown(article['article_content']))

## 4. Edit Article

Edit the title and content below. The content is loaded from the database.

In [ ]:
# ============================================
# EDIT THE TITLE (set to None to keep existing)
# ============================================
new_title = None  # or: new_title = "Your New Title Here"

# ============================================
# EDIT THE CONTENT BELOW
# ============================================
new_content = article['article_content'] if article else ""

# Show the content in an editable text area
print("Current content (copy, edit, and paste back):")
print("=" * 60)
print(new_content)

In [ ]:
# ============================================
# PASTE YOUR EDITED CONTENT HERE
# ============================================

new_content = """
# Your Edited Article Title

Your edited content goes here...

## Section 1

Content for section 1...

## Section 2

Content for section 2...

## Conclusion

Your conclusions...
"""

# Preview the edited content
print("Preview of edited content:")
print("=" * 60)
display(Markdown(new_content))

## 5. Save Changes

Run this cell to save your changes to the database.

In [ ]:
def update_article(article_id, new_content, new_title=None):
    """
    Update an article's content (and optionally title) in the database.
    
    Args:
        article_id: The ID of the article to update
        new_content: The new article content
        new_title: Optional new title (None to keep existing)
        
    Returns:
        True if successful, False otherwise
    """
    conn = get_connection()
    cursor = conn.cursor()
    
    try:
        if new_title:
            cursor.execute("""
                UPDATE articles
                SET article_content = ?, article_title = ?, updated_at = ?
                WHERE id = ?
            """, (new_content, new_title, datetime.now().isoformat(), article_id))
        else:
            cursor.execute("""
                UPDATE articles
                SET article_content = ?, updated_at = ?
                WHERE id = ?
            """, (new_content, datetime.now().isoformat(), article_id))
        
        conn.commit()
        print(f"Article {article_id} updated successfully!")
        return True
    except Exception as e:
        print(f"Error updating article: {e}")
        return False
    finally:
        conn.close()

# Confirm before saving
print(f"About to update Article ID: {ARTICLE_ID}")
print(f"New title: {new_title or '(keeping existing)'}")
print(f"Content length: {len(new_content)} characters")
print("\nRun the next cell to confirm and save.")

In [ ]:
# ============================================
# CONFIRM SAVE - Run this cell to save changes
# ============================================

# Safety check
if new_content.strip() == "" or new_content.strip().startswith("# Your Edited Article Title"):
    print("ERROR: You haven't edited the content yet!")
    print("Please edit the content in cell 4b first.")
else:
    success = update_article(ARTICLE_ID, new_content, new_title)
    if success:
        print("\nChanges saved to database.")

## 6. Verify Changes

Reload and view the updated article.

In [ ]:
# Reload and verify
updated_article = load_article(ARTICLE_ID)

if updated_article:
    print(f"Article ID: {updated_article['id']}")
    print(f"Title: {updated_article['article_title']}")
    print(f"Updated: {updated_article.get('updated_at', 'Never')}")
    print("\n" + "=" * 60 + "\n")
    display(Markdown(updated_article['article_content']))

## Utility Functions

Additional helper functions for article management.

In [ ]:
def get_article_videos(article_id):
    """Get all videos associated with an article."""
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT * FROM article_videos WHERE article_id = ?
    """, (article_id,))
    videos = cursor.fetchall()
    conn.close()
    return [dict(v) for v in videos]

def get_article_web_sources(article_id):
    """Get all web sources associated with an article."""
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT * FROM article_web_sources WHERE article_id = ?
    """, (article_id,))
    sources = cursor.fetchall()
    conn.close()
    return [dict(s) for s in sources]

def show_article_sources(article_id):
    """Show all sources (videos and web) for an article."""
    videos = get_article_videos(article_id)
    web_sources = get_article_web_sources(article_id)
    
    print(f"Sources for Article {article_id}")
    print("=" * 60)
    
    print(f"\nVideos ({len(videos)}):")
    for v in videos:
        print(f"  - {v.get('title', 'Unknown')[:50]}")
        print(f"    URL: {v.get('url', 'N/A')}")
    
    print(f"\nWeb Sources ({len(web_sources)}):")
    for w in web_sources:
        print(f"  - {w.get('title', 'Unknown')[:50]}")
        print(f"    URL: {w.get('url', 'N/A')}")

# Example: show sources for current article
# show_article_sources(ARTICLE_ID)

In [ ]:
def search_articles(query, limit=10):
    """Search articles by title or content."""
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT id, article_title, article_type, created_at
        FROM articles
        WHERE article_title LIKE ? OR article_content LIKE ?
        ORDER BY created_at DESC
        LIMIT ?
    """, (f"%{query}%", f"%{query}%", limit))
    articles = cursor.fetchall()
    conn.close()
    
    print(f"Search results for '{query}':")
    print("-" * 60)
    
    for a in articles:
        title = a['article_title'][:45] + "..." if len(a['article_title']) > 45 else a['article_title']
        print(f"ID {a['id']:>4} | {a['article_type'] or 'trending':>10} | {title}")
    
    return [dict(a) for a in articles]

# Example: search for articles
# search_articles("camera")

In [ ]:
def delete_article(article_id, confirm=False):
    """Delete an article and its associated data."""
    if not confirm:
        print(f"To delete article {article_id}, call: delete_article({article_id}, confirm=True)")
        return False
    
    conn = get_connection()
    cursor = conn.cursor()
    
    try:
        # Delete associated data first
        cursor.execute("DELETE FROM article_videos WHERE article_id = ?", (article_id,))
        cursor.execute("DELETE FROM article_web_sources WHERE article_id = ?", (article_id,))
        cursor.execute("DELETE FROM article_debug WHERE article_id = ?", (article_id,))
        cursor.execute("DELETE FROM articles WHERE id = ?", (article_id,))
        
        conn.commit()
        print(f"Article {article_id} deleted successfully!")
        return True
    except Exception as e:
        print(f"Error deleting article: {e}")
        return False
    finally:
        conn.close()

# Example: delete an article (uncomment to use)
# delete_article(123, confirm=True)